In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy.engine import Engine
from pathlib import Path
import yaml

In [2]:
# Настроим подключение к базе данных
CONFIG_PATH = "config.yaml"
with open(CONFIG_PATH, "r", encoding="utf-8") as config_file:
    CONFIG = yaml.load(config_file, Loader=yaml.FullLoader)

CON = create_engine(
    CONFIG['database_url']
)

def select(query: str, con: Engine = CON) -> pd.DataFrame:
    """
    Функция обобщённого sql-запроса к базе данных соцсети, содержащей три таблицы:
    1. Данные пользователей user
    2. Данные постов post
    3. Данные действий пользователей в соцсети feed
    """
    return pd.read_sql(query, con)

##### Загрузим данные из базы

In [3]:
# Посмотрим таблицу юзеров
q = """
SELECT *
FROM public.user
"""

user_data = select(q)
user_data.head()

AttributeError: 'OptionEngine' object has no attribute 'execute'

In [ ]:
user_data.shape

In [ ]:
# Теперь таблицу постов
q = """
SELECT *
FROM public.post
"""

post_data = select(q)
post_data.head()

In [ ]:
# Наконец посмотрим на таблицу взаимодействия пользователей и постов в соцсети
q = """
SELECT count(*)
FROM public.feed_data
"""

count_feed_data = select(q)
count_feed_data.head()

In [ ]:
# Как видно, в таблице 77 миллионов записей, что превышает наши вычислительные возможности.
# Для решения этой проблемы отсортируем и пронумеруем все действия юзеров с помощью оконной функции
# и отфильтруем ранние действия юзеров и будем рассматривать только не позднее 35го по счету.

q = """
WITH q1 (timestamp, user_id, post_id, action, target, num) AS 
(
    SELECT timestamp, user_id, post_id, action, target, ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY timestamp DESC) AS num
    FROM public.feed_data
    WHERE action = 'view'
)
SELECT timestamp, user_id, post_id, target
FROM q1
WHERE num <= 35
"""

feed_data = select(q)
feed_data.head()

In [ ]:
# Получаем уже приемлемое количество записей

feed_data.shape

##### Получим итоговый датасет для решения задачи

In [ ]:
# Смёрджим датафреймы для получения итогового датасета

data = feed_data.merge(
    user_data.rename(columns={'id': 'user_id'}), on='user_id'
).merge(
    post_data.rename(columns={'id': 'post_id'}), on='post_id'
)

data.head()

In [ ]:
data.shape

In [ ]:
# Сохраним датасеты для их дальнейшей обработки перед обучением модели

post_data.to_csv(CONFIG['datasets_folder'] + '/post_data.csv', index=False)
data.to_csv(CONFIG['datasets_folder'] + '/data.csv', index=False)